In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
import math
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
root.gROOT.SetBatch(True)

In [3]:
root.gSystem.Load("input/WriteEvent_C.so")

0

In [ ]:
# ---------------------------
# Input sample + generation settings
# ---------------------------

# Pick your input ROOT file (inside file_path)
fname = "star_jpsi_ee.root"   # example
#fname = "thermal.root"   # example

# Where the files live
file_path = "/media/yoren/T7_Shield/PHENIX/Dielectrons/helios/"  # adjust

# IMPORTANT: explicitly tell the notebook what kind of sample this is
#   - "helios_flat_pt" : generated flat in pT, needs pT reweight
#   - "thermal"        : already thermal, no pT reweight
SAMPLE_KIND = "helios_flat_pt"   # change to "thermal" when needed

# Generation ranges used when producing the sample (needed for correct normalization)
GEN_PT_MIN  = 0.0
GEN_PT_MAX  = 10.0
GEN_Y_MIN   = -1.0
GEN_Y_MAX   =  1.0
GEN_PHI_MIN = 0.0
GEN_PHI_MAX = 2.0*math.pi

DO_PT_REWEIGHT = (SAMPLE_KIND == "helios_flat_pt")

# For quick tests you can limit event count (set to None to run full)
nev_set = int(1e6)

print("DO_PT_REWEIGHT:", DO_PT_REWEIGHT)


DO_PT_REWEIGHT: True


In [5]:
# ---------------------------
# Analysis 'card' constants (ported from your C++ snippet)
# ---------------------------

dNdy_pp = [42.2*95/257, 42.2*11/257, 8.6*42/257, 4.3, 4.3, 0.92, 0.93, 0.93, 1.77e-5, 0.133/1000/42.2, 0.12, 0.01, 10]
T_pp    = [0.113, 0.113, 0.139, 0.139, 0.110, 0.110, 0.139, 0.139, 0.149, 0.164, 0.3, 0.3, 0.3]
n_pp    = [9.57, 9.57, 10.8, 9.7, 9.78, 9.78, 10.8, 10.8, 12.3, 14, 0, 0, 0]
m_pp    = [0.1349766, 0.547862, 0.77526, 0.78265, 0.78265, 0.95778, 1.019461, 1.019461, 3.096916, 3.68609, -1.019461, -9.46030, 1]
Ncolls  = [771, 282.4, 82.6, 12.1, 3]
Br_to_ee = [1.174e-2, 6.9e-3, 4.73e-5, 7.38e-5, 7.7e-4, 4.73e-4, 2.98e-4, 1.08e-4, 5.94e-2, 7.9e-3, 1, 1, 1]
hith_pt_ratio = [1.0, 0.48, 1.0, 0.78, 0.78, 0.25, 0.4, 0.4, 0.01, 0.5, 1, 1, 1]
pi0_high_pt_ratio = [0.00623224, 0.00228273, 0.000667683, 9.78082e-05, 2.425e-05]
part_names = ["pi0","eta","rho->ee","omega","omega->pi0ee","eta'->gamma ee","phi->ee","phi->etaee","jpsi","psi2s","ccbar","bbbar","qgp"]

# Hagedorn parameters (choose table + centrality)
# hag_params: 5 bins = [0–20, 20–40, 40–60, 60–80, 80–93]
hag_params = [
  [1166.0,   0.5457,  0.17865, 0.7470,  8.311 ],
  [643.0,    0.4717,  0.14155, 0.7502,  8.316 ],
  [297.85,   0.42765, 0.1124,  0.73215, 8.241 ],
  [93.65,    0.3633,  0.0453,  0.7501,  8.348 ],
  [51.1,     0.2470,  0.0619,  0.7101,  8.459 ],
]

# Select centrality bin for the target pi0 spectrum (0..4)
cent_bin = 0  # 0–20%
c, a, b, p0, n = hag_params[cent_bin]

print("Using centrality bin", cent_bin, "hag params:", (c,a,b,p0,n))


Using centrality bin 0 hag params: (1166.0, 0.5457, 0.17865, 0.747, 8.311)


In [6]:
# Output file for histograms
outfilename = "output/hists/" + fname.replace(".root", "_hists.root")
print("Output:", outfilename)


Output: output/hists/thermal_hists.root


In [7]:
f = root.TFile.Open(file_path+fname, "READ")
assert f and not f.IsZombie(), f"Could not open {fname}"

T = f.Get("T")
assert T, "Could not find TTree named 'T'"

print("Entries:", T.GetEntries())
T.Print()

Entries: 1000000
******************************************************************************
*Tree    :T         : Thermal parent -> e+e- (WriteEvent)                    *
*Entries :  1000000 : Total =       320494186 bytes  File  Size =   89280061 *
*        :          : Tree compression factor =   3.59                       *
******************************************************************************
*Branch  :MyEvent                                                            *
*Entries :  1000000 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :fUniqueID : UInt_t                                                 *
*Entries :  1000000 : Total  Size=    4004603 bytes  File Size  =      22307 *
*Baskets :       46 : Basket Size=     879104 bytes  Compression= 179.47     *
*............................................................................*
*Br    1 :fBits     : UInt_t       

In [8]:
def get_tracks_from_event(MyEvent):
    """Return list of WriteTrack (by value) from a WriteEvent."""
    n = int(MyEvent.GetNEntries())
    return [MyEvent.GetWriteTrack(i) for i in range(n)]


In [9]:
def track_info(trk):
    """Extract common fields from WriteTrack."""
    pid = int(trk.GetID())
    isFinal = int(trk.GetFinal())
    br = float(trk.GetBranch())
    parent = int(trk.GetParentIndex())
    w = float(trk.GetWeight())
    p4 = root.TLorentzVector(float(trk.GetPx()), float(trk.GetPy()), float(trk.GetPz()), float(trk.GetEnergy()))
    return dict(pid=pid, isFinal=isFinal, br=br, parent=parent, w=w, p4=p4)


In [10]:
# ---------------------------
# pT reweighting (accurate): w(pt) = p_target(pt) / p_gen(pt)
 # For HELIOS flat-pt generation: p_gen = 1/(ptmax-ptmin)
# Target pT PDF: normalized Hagedorn on [GEN_PT_MIN, GEN_PT_MAX]
# ---------------------------

# Hagedorn functional form from your C++ snippet:
#   f(pt) = c*pt / ( exp(-a*pt - b*pt^2) + pt/p0 )^n
f_hag = root.TF1(
    "f_hag",
    "[0]*x / pow( exp(-[1]*x - [2]*x*x) + x/[3], [4] )",
    GEN_PT_MIN, GEN_PT_MAX
)
f_hag.SetParameters(c, a, b, p0, n)

I_hag = f_hag.Integral(GEN_PT_MIN, GEN_PT_MAX)
assert I_hag > 0, "Hagedorn integral non-positive"

GEN_PT_WIDTH = (GEN_PT_MAX - GEN_PT_MIN)
assert GEN_PT_WIDTH > 0

def pT_pdf_target(pt):
    if pt < GEN_PT_MIN or pt > GEN_PT_MAX:
        return 0.0
    return float(f_hag.Eval(pt) / I_hag)

def pT_pdf_gen(pt):
    # HELIOS generation is flat in pT on [GEN_PT_MIN, GEN_PT_MAX]
    if pt < GEN_PT_MIN or pt > GEN_PT_MAX:
        return 0.0
    return 1.0 / GEN_PT_WIDTH

def pt_reweight(pt):
    """Return p_target(pt)/p_gen(pt). For thermal sample, returns 1."""
    if not DO_PT_REWEIGHT:
        return 1.0
    gen = pT_pdf_gen(pt)
    if gen <= 0:
        return 0.0
    return pT_pdf_target(pt) / gen

print("Hagedorn norm integral:", I_hag)
for test_pt in [0.2, 1.0, 2.0, 5.0, 9.0]:
    print("pt", test_pt, "w", pt_reweight(test_pt))


Hagedorn norm integral: 38.646911234194256
pt 0.2 w 17.834552107360224
pt 1.0 w 2.048977798315067
pt 2.0 w 0.10255163587208117
pt 5.0 w 0.00020709846964796577
pt 9.0 w 2.8202284853034953e-06


In [11]:
h2_m_pt = root.TH2D(
    "h2_m_pt",
    "e^{+}e^{-}: M_{ee} vs p_{T};p_{T} (GeV/c);M_{ee} (GeV/c^{2})",
    200, 0.0, 10.0,
    200, 0.0, 5.0
)

h_mult_final = root.TH1D("h_mult_final", "Final-state multiplicity;N_{final};Events", 20, -0.5, 19.5)
h_br_final   = root.TH1D("h_br_final",   "Final track branch br;br;Counts",            50, -0.5, 49.5)
h1_pt_original = root.TH1D("h1_pt_original", "Original pT distribution;p_{T} (GeV/c);Counts", 100, 0.0, 10.0)
h1_pt_mother_reweighted = root.TH1D("h1_pt_mother_reweighted", "Mother pT distribution (reweighted);p_{T} (GeV/c);Counts", 100, 0.0, 10.0)

In [12]:
nentries = int(T.GetEntries())
if nev_set is not None:
    nentries = min(nentries, int(nev_set))

for iev in range(nentries):
    T.GetEntry(iev)
    ev = T.MyEvent

    ntrk = int(ev.GetNEntries())

    mother_p4 = None
    finals = []  # list of dicts

    for i in range(ntrk):
        trk = ev.GetWriteTrack(i)
        info = track_info(trk)

        if info["isFinal"] == 0:
            # choose the primary mother (ParentIndex < 0 is typical)
            if mother_p4 is None and info["parent"] < 0:
                mother_p4 = info["p4"]
            continue

        # final track
        finals.append(info)
        h_br_final.Fill(info["br"])

    h_mult_final.Fill(len(finals))

    eminus = [t for t in finals if t["pid"] == 11]
    eplus  = [t for t in finals if t["pid"] == -11]

    # mother pT for reweighting (recommended for flat-pt parent generation)
    pt_mother = mother_p4.Pt() if mother_p4 is not None else None
    h1_pt_original.Fill(pt_mother if pt_mother is not None else -1)
    if DO_PT_REWEIGHT and pt_mother is not None:
        h1_pt_mother_reweighted.Fill(pt_mother, pt_reweight(pt_mother))
    for e1 in eminus:
        for e2 in eplus:
            pair = e1["p4"] + e2["p4"]
            mee  = pair.M()
            pt_pair = pair.Pt()

            # Base weight from file (branching weights)
            w = float(e1["w"] * e2["w"])

            # pT spectrum reweight (HELIOS flat only)
            if DO_PT_REWEIGHT:
                if pt_mother is None:
                    continue
                w *= pt_reweight(pt_mother)

            h2_m_pt.Fill(pt_pair, mee, w)

    if iev > 0 and (iev % 1000000 == 0):
        print("Processed", iev, "/", nentries)


In [13]:
c3 = root.TCanvas("c3","c3",800,600)
h1_pt_original.Draw()
c3.Draw()
c4 = root.TCanvas("c4","c4",800,600)
h1_pt_mother_reweighted.Draw()
c4.Draw()

In [14]:
c1 = root.TCanvas("c1","c1",900,700)
h2_m_pt.Draw("COLZ")
c1.SetLogz(True)
c1.SaveAs("output/hists/mee_vs_pt.png")

c2 = root.TCanvas("c2","c2",800,600)
h_mult_final.Draw()
c2.SaveAs("output/hists/mult_final.png")

c3 = root.TCanvas("c3","c3",800,600)
h_br_final.Draw()
c3.SaveAs("output/hists/br_final.png")

In [16]:
c5 = root.TCanvas("c5","c5",800,600)
mass_hist = h2_m_pt.ProjectionY("mass_hist")
mass_hist.Draw()
c5.Draw()

In [15]:
out = root.TFile(outfilename, "RECREATE")
h2_m_pt.Write()
h_mult_final.Write()
h_br_final.Write()
out.Close()

print("Wrote pair_hists.root")

Wrote pair_hists.root
